In [1]:
!pip install --quiet mxnet

In [5]:
import warnings
warnings.filterwarnings('ignore')
import mxnet as mx
import fetch.fetch as fetch
from mxnet import nd, autograd
mx.random.seed(1)

In [6]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))
ga = df[df._id.str.contains('ge')]
tweets = df[df._id.str.contains('tw')]
tweets = tweets[tweets.body.str.contains('travel ban') == False]

In [326]:
import pandas as pd

from sklearn.preprocessing import scale

housing = pd.read_csv('mxnet-data/kc_house_data.csv')

# housing set as years from min... 
# housing['yr_built'] -= housing['yr_built'].min()
X = housing[['sqft_living', 'sqft_lot', 'bathrooms', 'yr_built', 'bedrooms']].as_matrix()
X = scale(X)

# log prices is probably what we care about
y = scale(np.log(housing.price.as_matrix()))

train_data = mx.gluon.data.DataLoader(mx.gluon.data.ArrayDataset(X, y), batch_size=4)

In [54]:
w = nd.random_normal(shape=(X.shape[1], 1), dtype=float)
b = nd.random_normal(shape=1, dtype=float)
params = [w, b]

for param in params:
    param.attach_grad()

def net(X, params):
    return mx.nd.dot(X, params[0]) + params[1]

def square_loss(yhat, y):
    return nd.mean((yhat - y) ** 2)

def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [41]:
params

[
 [[-0.2611377 ]
  [-0.20703205]
  [-1.24573995]
  [-1.96754877]
  [ 0.34168572]]
 <NDArray 5x1 @cpu(0)>, 
 [-0.92148392]
 <NDArray 1 @cpu(0)>]

In [112]:
epochs = 5
ctx = mx.cpu()
learning_rate = .001
smoothing_constant = .01


for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx).reshape((-1, 1))
        with autograd.record():
            output = net(data, params)
            loss = square_loss(output, label)
        loss.backward()
        SGD(params, learning_rate)
        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

        if i == 2000:
            print("Epoch %s, batch %s. Moving avg of loss: %s" % (e, i, curr_loss))

Epoch 0, batch 2000. Moving avg of loss: 0.291221741238


Epoch 1, batch 2000. Moving avg of loss: 0.291221741238


Epoch 2, batch 2000. Moving avg of loss: 0.291221741238


Epoch 3, batch 2000. Moving avg of loss: 0.291221741238


Epoch 4, batch 2000. Moving avg of loss: 0.291221741238


In [168]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)
print(model.coef_, model.intercept_)
print(model.score(X, y))

model.coef_ = params[0].asnumpy().reshape((5))
model.intercept_ = params[1].asnumpy()[0]
model.score(X, y)

[ 0.6755889  -0.0206198   0.22149048 -0.22595775 -0.12480304] -1.90226971031e-15
0.528605581373


0.50899433708531561

In [160]:
from mxnet import gluon
batch_size = 4

ctx = mx.cpu()
net = gluon.nn.Sequential()
square_loss = gluon.loss.L2Loss()

with net.name_scope():
    net.add(gluon.nn.Dense(1))

net.collect_params().setattr('dtype', 'float64')
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=ctx)

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

net.collect_params()

sequential23_ (
  Parameter sequential23_dense0_weight (shape=(1, 0), dtype=float64)
  Parameter sequential23_dense0_bias (shape=(1,), dtype=float64)
)

In [161]:
epochs = 2
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(4)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    print("Epoch %s. Moving avg of MSE: %s" % (e, moving_loss))

Epoch 0. Moving avg of MSE: 0.209680460193


Epoch 1. Moving avg of MSE: 0.209680460193


In [172]:
[p.data() for k,p in net.collect_params().items()]

[
 [[ 0.6366786   0.07320998  0.20614176  0.01330564 -0.17872601]]
 <NDArray 1x5 @cpu(0)>, 
 [ 0.04198348]
 <NDArray 1 @cpu(0)>]

In [ ]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))
df['label'] = [1 if l == 'accepted' else 0 for l in df.label]

In [186]:
from modelling.models import *
word_count = create_word_count(df)

In [ ]:
from sklearn.model_selection import train_test_split

y = df.label.as_matrix()
X = np.asarray(word_count.todense())
X_train, X_test, y_train, y_test
train_data = mx.gluon.data.DataLoader(mx.gluon.data.ArrayDataset(X, y), batch_size=4)

In [442]:
w = nd.random_normal(shape=(X.shape[1], 1), dtype = 'float64')
b = nd.random_normal(shape=1, dtype = 'float64')
params = [w, b]

for param in params:
    param.attach_grad()

def logistic(x):
    return 1 / (1 + nd.exp(-x))

def net(X):
    y_linear = nd.dot(X, w) + b
    yhat = logistic(y_linear)
    return yhat

def cross_entropy(yhat, y):
    like = (1-y)*nd.log(1 - yhat) + y*nd.log(yhat) 
    return -like

def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [452]:
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.round(output).astype(int).reshape((4,))
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator.asscalar() / denominator)

In [414]:
evaluate_accuracy(train_data, net)

0.62776484284051226

In [ ]:
epochs = 10
moving_loss = 0.
learning_rate = .01
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx).astype(float)
        with autograd.record():
            output = net(data)[:,0]
            loss = cross_entropy(output, label)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s" % (e, moving_loss, train_accuracy))